In [1]:
import pandas as pd
import psycopg2
from sklearn.preprocessing import MinMaxScaler

In [2]:
try:
    # Establish a connection to the PostgreSQL database
    connection = psycopg2.connect(
        host = "localhost",
        dbname = "nfl",
        user = "postgres",
        password = "Plenoir2002!", # Include correct password
        port = 5432
    )
    print("Connected to PostgreSQL database.")
except (Exception, psycopg2.Error) as error:
    print("Error connecting to PostgreSQL database: ", error)

# Create a cursor object to interact with the database
cursor = connection.cursor()

Connected to PostgreSQL database.


In [3]:
# Load and preprocess dataframe
nfl_df = pd.read_csv('nfl_games.csv')  # Replace with your data file path

# Specify the number of games to average
num_games_average = 3

# Define the offense/defense statistics to keep
offensive_stats = nfl_df.loc[:, 'Score':'FGA']
defensive_stats = nfl_df.loc[:, 'Score_opp':'FGA_opp']

# Get the column names as separate lists
o_col_names = list(offensive_stats.columns)
d_col_names = list(defensive_stats.columns)

# Create empty pandas DF
col_names = o_col_names + d_col_names
training_df = pd.DataFrame(columns=col_names)

In [4]:
# Function to calculate the rolling average for a team
def calculate_rolling_average(offensive_team, defensive_opponent, current_date, num_games_avg):
    # SQL query to fetch the previous games for the offensive team (up to num_games_avg)
    offense_query = f"""
    SELECT {', '.join(offensive_stats)}
    FROM {offensive_team}
    WHERE date < %s
    ORDER BY date DESC
    LIMIT %s;
    """
    cursor.execute(offense_query, (current_date, num_games_avg))
    offense_games = cursor.fetchall()

    # SQL query to fetch the previous games for the defensive opponent (up to num_games_avg)
    defense_query = f"""
    SELECT {', '.join(defensive_stats)}
    FROM {defensive_opponent}
    WHERE date < %s
    ORDER BY date DESC
    LIMIT %s;
    """
    cursor.execute(defense_query, (current_date, num_games_avg))
    defense_games = cursor.fetchall()

    # Concatenate the offensive and defensive values from the previous games (rounded to 2 decimal pts)
    game_avgs = [round(sum(values) / len(values), 2) for values in zip(*offense_games)] + [round(sum(values) / len(values), 2) for values in zip(*defense_games)]

    return game_avgs

In [5]:
# 2018 Week 4: '2018-09-27'
# Week 5: '2018-10-04'
# Week 6: '2018-10-11'

start_date = '2018-09-27'

# Iterate through your data and populate the DataFrame from the start date
for index, row in nfl_df.iterrows():
    team = row['Tm']
    opponent = row['Tm_opp']
    current_date = row['Date']
    
    # Account for the beginning of the data set (no prior games to average)
    if current_date >= start_date:
        
        avg_values = calculate_rolling_average(team, opponent, current_date, num_games_average)
        training_df.loc[index] = avg_values

In [6]:
# Close the cursor and the database connection
cursor.close()
connection.close()

In [7]:
training_df.columns = [f'{col}_L{num_games_average}' for col in training_df.columns]

# Filter nfl_df
filtered_nfl_df = nfl_df[nfl_df['Date'] > start_date]

columns_to_include = ['Tm', 'Score', 'Home', 'Date']
filtered_nfl_df = filtered_nfl_df[columns_to_include]

merged_df = pd.concat([filtered_nfl_df, training_df], axis=1)
merged_df = merged_df.dropna(subset=['Tm'])

In [8]:
merged_df

,Tm,Score,Home,Date,Score_L3,FrDwns_L3,TotYds_L3,T_O_L3,Pen_L3,Pen_Yds_L3,...,QB_Prss_opp_L3,Rush_Att_opp_L3,Rush_Yds_opp_L3,Rush_TD_opp_L3,Rush_YAC_opp_L3,Rush_YPC_opp_L3,XPM_opp_L3,XPA_opp_L3,FGM_opp_L3,FGA_opp_L3
98,LVR,45.0,1.0,2018-09-30,17.33,22.00,400.67,1.67,7.67,74.33,...,10.33,29.33,109.33,1.33,60.67,3.60,2.00,2.00,1.00,1.67
99,NYJ,12.0,0.0,2018-09-30,25.67,17.67,326.33,2.67,6.67,51.33,...,10.33,27.33,115.33,0.33,75.33,4.23,0.67,0.67,2.67,3.33
100,BAL,26.0,0.0,2018-09-30,32.33,24.67,378.67,1.33,7.67,60.33,...,16.33,26.33,122.33,0.67,42.67,4.57,4.00,4.00,0.67,1.00
101,SFO,27.0,0.0,2018-09-30,24.33,23.00,359.67,1.33,8.67,84.67,...,9.67,28.00,120.33,0.67,49.33,4.20,4.00,4.00,1.00,1.33
102,NOR,33.0,0.0,2018-09-30,34.67,25.33,428.00,1.33,7.67,83.33,...,9.67,24.00,111.33,0.33,41.33,4.50,1.67,1.67,2.33,2.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2739,CIN,20.0,0.0,2023-01-29,26.00,21.00,301.00,0.67,4.00,28.67,...,12.67,22.67,120.00,1.00,22.33,5.50,2.00,2.00,1.67,1.67
2740,SFO,7.0,0.0,2023-01-29,32.67,22.00,376.00,0.33,2.67,22.67,...,11.00,25.33,125.33,1.00,45.00,5.23,1.33,1.33,1.00,1.00
2741,PHI,31.0,1.0,2023-01-29,23.33,20.00,357.00,0.67,5.67,34.67,...,4.33,22.33,80.33,0.67,22.00,3.57,1.00,1.67,1.00,1.00
2742,PHI,35.0,1.0,2023-02-12,30.33,24.67,342.33,0.33,4.33,29.67,...,12.33,20.33,104.67,0.67,27.33,5.27,1.67,1.67,2.00,2.00


In [9]:
def normalize_dataframe(df, columns_to_normalize):
    # Create a MinMaxScaler object
    scaler = MinMaxScaler()

    # Fit the scaler to the specified columns and transform the data
    df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])

    return df, scaler

In [10]:
# Define the range of columns to normalize
start_column = 'Score_L3'
end_column = 'FGA_opp_L3'

columns_to_normalize = merged_df.columns[merged_df.columns.get_loc(start_column):merged_df.columns.get_loc(end_column) + 1]
normalized_df, scaler = normalize_dataframe(merged_df.copy(), columns_to_normalize)

In [11]:
normalized_df

,Tm,Score,Home,Date,Score_L3,FrDwns_L3,TotYds_L3,T_O_L3,Pen_L3,Pen_Yds_L3,...,QB_Prss_opp_L3,Rush_Att_opp_L3,Rush_Yds_opp_L3,Rush_TD_opp_L3,Rush_YAC_opp_L3,Rush_YPC_opp_L3,XPM_opp_L3,XPA_opp_L3,FGM_opp_L3,FGA_opp_L3
98,LVR,45.0,1.0,2018-09-30,0.281228,0.567144,0.607914,0.4175,0.576364,0.583069,...,0.491053,0.525152,0.341121,0.362398,0.443849,0.293165,0.333333,0.299850,0.230947,0.385681
99,NYJ,12.0,0.0,2018-09-30,0.476682,0.373321,0.416153,0.6675,0.485455,0.374603,...,0.491053,0.464545,0.369159,0.089918,0.570589,0.406475,0.111667,0.100450,0.616628,0.769053
100,BAL,26.0,0.0,2018-09-30,0.632763,0.686661,0.551165,0.3325,0.576364,0.456177,...,0.806842,0.434242,0.401869,0.182561,0.288234,0.467626,0.666667,0.599700,0.154734,0.230947
101,SFO,27.0,0.0,2018-09-30,0.445278,0.611907,0.502154,0.3325,0.667273,0.676788,...,0.456316,0.484848,0.392523,0.182561,0.345811,0.401079,0.666667,0.599700,0.230947,0.307159
102,NOR,33.0,0.0,2018-09-30,0.687603,0.716204,0.678412,0.3325,0.576364,0.664642,...,0.456316,0.363636,0.350467,0.089918,0.276649,0.455036,0.278333,0.250375,0.538106,0.538106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2739,CIN,20.0,0.0,2023-01-29,0.484415,0.522381,0.350814,0.1675,0.242727,0.169220,...,0.614211,0.323333,0.390981,0.272480,0.112389,0.634892,0.333333,0.299850,0.385681,0.385681
2740,SFO,7.0,0.0,2023-01-29,0.640731,0.567144,0.544277,0.0825,0.121818,0.114837,...,0.526316,0.403939,0.415888,0.272480,0.308377,0.586331,0.221667,0.199400,0.230947,0.230947
2741,PHI,31.0,1.0,2023-01-29,0.421842,0.477619,0.495267,0.1675,0.394545,0.223602,...,0.175263,0.313030,0.205607,0.182561,0.109536,0.287770,0.166667,0.250375,0.230947,0.230947
2742,PHI,35.0,1.0,2023-02-12,0.585892,0.686661,0.457425,0.0825,0.272727,0.178283,...,0.596316,0.252424,0.319346,0.182561,0.155615,0.593525,0.278333,0.250375,0.461894,0.461894


In [12]:
normalized_df.to_csv("nfl_training_data.csv")